# Visualising Calibration data

This example shows how we can query the calibration information of Helmi and visualise it using matplotlib. However, we need to understand what calibration is and why the calibration information is important


## What is Calibration
Calibration is an important process that involves fine-tuning the parameters of a quantum computer to ensure the stability and accuracy of quantum operations. The calibration procedures varies depending on the architecture of the quantum computers, the type of qubits used and the error correction techniques implemented. Generally, Calibration may involve the following
- Gate calibration - adjusting and measuring the pulse duration, amplitudes and fidelity of the quantum gates, 
- Noise Mitigation - identifying and mitigating environmental noise,
- Qubit Calibration - measuring and understanding the coherence times of the qubits. The coherence time is a measure of how long a qubit can retain its information.

## Why is Calibration Important

Quantum computers, unlike their classical counterparts,  operate on the principles of quantum mechanics, as a result they are highly sensitive to environmental interference ( such as changes in temperature, electromagnectic fields, vibrations) and hardware limitations or imperfections. Calibration ensures that the system is stable and can return accurate results. 

The result from the calibration data provides users with information such as coherence times and fidelity of the gates which allows them to optimize their quantum circuits. Overall, it gives an idea of the current status of Helmi. Here is a brief description of figures received from the calibration data: [Figures-of-merit](https://docs.csc.fi/computing/quantum-computing/helmi/running-on-helmi/#figures-of-merit).

## Setup

This notebook uses the following requirements

```text
qiskit-iqm==10.4 (can easily be switched with cirq-iqm)
iqm-client==13.0
matplotlib
```

In [ ]:
import os
import json
import requests
import matplotlib.pyplot as plt
from iqm_client import IQMClient
from qiskit_iqm import IQMProvider

Connect to the backend

In [ ]:
provider = IQMProvider("https://qc.vtt.fi/cocos")
backend = provider.get_backend()

## Querying Calibration Data

Here we define a function that has 3 parameters. 

- client
- calibration_set_id: ID of the calibration set to use. This is optional and the function will by default query the latest calibration set if the id is not provided.
- filename: if provided, it specifies the file to save the response


In [ ]:
def get_calibration_data(client: IQMClient, calibration_set_id=None, filename: str = None):
    headers = {'User-Agent': client._signature}
    bearer_token = client._get_bearer_token()
    headers['Authorization'] = bearer_token if bearer_token else ''
    url = os.path.join(client._base_url, 'calibration')
    if calibration_set_id:
        url = os.path.join(url, calibration_set_id)
    response = requests.get(url, headers=headers)
    
    # returns the status of the response
    response.raise_for_status()
    data = response.json()
    data_str = json.dumps(data, indent=4)

    if filename:
        with open(filename, 'w') as f:
            f.write(data_str)
        print(f"Data saved to {filename}")

    return data

In [ ]:
calibration_data = get_calibration_data(backend.client)

We can print the calibration data that our function returns

In [ ]:
print(calibration_data)

The printout give a lot of values that might be difficult to understand at a glance. We can use the quality metrics set(figures of merit) along with matplotlib to visualise the calibration information. 

The plot_metrics function iterates over a json data and maps the keys to the correct metric.

In [ ]:
def plot_metrics(metric: str,
                 title: str,
                 ylabel: str,
                 xlabel: str,
                 data: dict,
                 limits: list = []
                 ):
    values = []
    labels = []
    # Iterate over the JSON data and collect values and labels based on the metric
    for key, metric_data in data['metrics'].items():
        if key.endswith(metric):
            values.append(float(metric_data['value']))
            # Extract the qubit label from the key
            labels.append(key.split('.')[0])

    # Check if values were found for the given metric
    if not values:
        return f"{metric} not in quality metrics set!"

    # Set the width and gap between the bars
    bar_width = 0.2

    # Calculate the positions of the bars
    positions = range(len(values))

    # Plot the values with labels
    plt.bar(positions, values, width=bar_width, tick_label=labels)

    if len(limits) == 2:
        plt.ylim(limits)

    plt.grid(axis='y')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.xticks(rotation=90)
    plt.show()


Here we call the plot_metrics function with the calibration data as a parameter

In [ ]:
plot_metrics(metric='cz_gate_fidelity', title='CZ Gate Fidelities',
             xlabel='Qubit pairs', ylabel='Fidelities', data=calibration_data, limits=[0.6,1])

## Summary

In this notebook, we have briefly introduced the concept of calibration, we have also demonstrated how to query calibration data from Helmi and visualise it using matplotlib.

## Additional Reading

- [Superconducting qubits](https://pennylane.ai/qml/demos/tutorial_sc_qubits).
- [Calibration experiments with qiskit](https://qiskit.org/ecosystem/experiments/tutorials/calibrations.html).
- [How to automate calibration](https://docs.q-ctrl.com/boulder-opal/user-guides/how-to-automate-calibration-of-control-hardware).